In [1]:
!pip install SentencePiece

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import transformers
from torch.optim import Adam
from tqdm import tqdm

In [3]:
# prompt: mount from google drive

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
%cd drive/MyDrive/dlt_results/

/content/drive/MyDrive/dlt_results


In [5]:
import torch
from torch.utils.data import Dataset, DataLoader, Subset
from transformers import T5Tokenizer, T5Model, T5ForConditionalGeneration, T5TokenizerFast

In [6]:
from utils import FinQA_Dataset

In [22]:
TOKENIZER = T5Tokenizer.from_pretrained("t5-base")
MODEL = T5ForConditionalGeneration.from_pretrained("./finqa_finetune_t5.pth/")
BATCH_SIZE = 8
TESTING = False

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:240: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [23]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE

device(type='cuda')

In [24]:
MODEL.to(DEVICE);

### Read Test and do inference

In [25]:
finqa_test = torch.load('./finqa_test.pth')
len(finqa_test)

1108

In [26]:
if TESTING:
    test_loader = DataLoader(Subset(finqa_test, range(100)), batch_size=BATCH_SIZE)
else:
    test_loader = DataLoader(finqa_test, batch_size=BATCH_SIZE)

In [27]:
TOKENIZER.decode([x for x in finqa_test[0]['labels'] if x!= -100], skip_special_tokens=True)

'Step 1: Subtract 5735 from 5829. This gives the result: 94'

In [28]:
MODEL.eval();
generated_texts = []
for batch in tqdm(test_loader, desc="Generation batches"):
    input_ids = batch["input_ids"].to(DEVICE)
    attention_mask = batch["attention_mask"].to(DEVICE)
    decoder_attention_mask = batch["decoder_attention_mask"].to(DEVICE)
    with torch.no_grad():
        # Use the `generate` method for text generation
        generated_output = MODEL.generate(
            input_ids=input_ids,
            max_length=128,  # Set the desired maximum length for generated text
            num_beams=5,    # You can adjust the number of beams for beam search
            temperature=0.8  # You can adjust the temperature for sampling,
            )
        for output_i in generated_output:
          generated_text = TOKENIZER.decode([x for x in output_i if x!= -100], skip_special_tokens=True)
          generated_texts.append(generated_text)

Generation batches:   0%|          | 0/139 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Generation batches:  63%|██████▎   | 88/139 [04:25<02:08,  2.52s/it]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Generation batches: 100%|██████████| 139/139 [06:53<00:00,  2.97s/it]


In [29]:
len(generated_texts)

1108

In [30]:
import pickle
file_path = './generated_sentences_test.pkl'
# Open the file in binary read mode and use pickle.load() to load the list
with open(file_path, 'wb') as file:
    pickle.dump(generated_texts, file)